In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

from typing import List

from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

from scipy.integrate import simpson

In [7]:
S0 = 8.0
D0 = 0.0
YEARS = 10
PI = np.pi
DELTA = 1/365

# el ingreso diario generado es 1000 ∗ Si dolares si el molino esta prendido ese dia
PRICE = 1000
# La probabilidad de que el molino se rompa en el dia i, si estaba activo en el dia i −1, es 0.25 ∗ S(i)2∆.
BREAK_PROB = 0.25
# Si el molino falla un dıa, ese dia no estara activo y se arreglara el mismo dıa a un costo de 200 mil dolares
BREAK_COST = 200000
# El acto de apagar el molino cuesta 1000 dolares cada vez. Prenderlo no cuesta nada.
TURN_OFF_COST = 1000
TURN_ON_COST = 0
# La tasa de inter ́es es del 5% anual con capitalizacion diaria (el cashflow en dia i se descuenta como e−0.05i/365)
INTEREST_RATE = 0.05

POWER_GENERATION = 100  # Generación de energía por día cuando está en funcionamiento
PRICE_PER_KWH = 0.05  # Precio por kWh

N=365

kappa, sigma = 109.25770145796069, 16.047720266718766
beta, gamma = 48.376849376532675, 10.091946894398104

# Define la función theta
def theta2(i, a, b, c, d):
    return a + b * np.cos(c * i + d)

def proba_romperse(S):
    return BREAK_PROB * np.array(S) ** 2 * DELTA

def inversa_proba_romperse(p):
    return np.sqrt( p / (BREAK_PROB*DELTA))

def ingreso(S:List[float], isOn:List[bool],seRompio:List[bool],costo_arreglo = BREAK_COST):
    break_cost = costo_arreglo * np.logical_and(isOn,seRompio)
    return PRICE * np.array(S) * np.logical_and(isOn,np.logical_not(np.array(seRompio)))  - break_cost

def nextStateIsOn(S:List[float], isOn:List[bool],k): #     TT , TF , FT , FF
    turn_off_candidate = np.array(S) >= k
    prender = ~np.logical_and(isOn , turn_off_candidate)# TT = 0, TF = 0, FT = 0, FF = 1
    apagar = np.logical_and(isOn , turn_off_candidate)  # TT = 1, TF = 0, FT = 0, FF = 0

    costo_prender_total = prender * TURN_ON_COST
    costo_apagar_total = apagar * TURN_OFF_COST

    nextIsOn = ~turn_off_candidate
    
    return nextIsOn, costo_prender_total + costo_apagar_total

def seRompio(S:List[float],isOn:List[bool]):
    return np.logical_and(np.random.uniform(0,1,size=(np.array(S).shape)) < proba_romperse(S),isOn)


def ganancia(molinoNorte, molinoSur,k,seed=0,costoArreglo = BREAK_COST):
    """M simulaciones de molino norte y M simulaciones de molino sur y n dias para cada simulacion, k el umbral elegido 
    
    """
    if seed != 0:
        np.random.seed(seed)
    #si el molino esta prendido en el dia i
    isOnNorte = np.zeros_like(molinoNorte) + 1
    isOnSur   = np.zeros_like(molinoSur) + 1

    #calculamos dia a dia ganancia acumulada
    gananciaNorte = np.zeros_like(molinoNorte)
    gananciaSur = np.zeros_like(molinoSur)

    #si hay una rotura en el dia i, independiente a prendido o apagado
    roturaNorte = np.zeros_like(molinoNorte)
    roturaSur = np.zeros_like(molinoSur)
    
    # recorremos dia a dia 
    for t in range(1,molinoNorte.shape[1]-1):
        #para el norte:
        #chequeamos si ayer estuvo prendido o apagado segun el viento de hoy 
        #me fijo si hoy se rompio (viento de hoy + prendido y apagado)
        #calculamos el ingreso(viento + prendido si esta roto)
        #maximizamos beneficios 
        isOnNorte[:,t],costosNorte_t = nextStateIsOn(molinoNorte[:,t],isOnNorte[:,t-1],k)
        roturaNorte[:,t] = seRompio(molinoNorte[:,t],isOnNorte[:,t])
        ingresoNorte_t = ingreso(molinoNorte[:,t],isOnNorte[:,t],roturaNorte[:,t],costo_arreglo=costoArreglo)
        gananciaNorte[:,t] =  gananciaNorte[:,t-1] + ingresoNorte_t - costosNorte_t

        #para el sur:
        
        isOnSur[:,t],costosSur_t = nextStateIsOn(molinoSur[:,t],isOnSur[:,t-1],k)
        roturaSur[:,t] = seRompio(molinoSur[:,t],isOnSur[:,t])
        ingresoSur_t = ingreso(molinoSur[:,t],isOnSur[:,t],roturaSur[:,t],costo_arreglo=costoArreglo)
        gananciaSur[:,t] = gananciaSur[:,t-1] +  ingresoSur_t - costosSur_t
        
    
    return gananciaNorte, gananciaSur, isOnNorte, isOnSur, roturaNorte, roturaSur

#INTEREST_RATE = 0.05  # tasa de interés de ejemplo

def van(ganancia,cumSum:bool=True):
    dia_a_dia = np.diff(ganancia, axis=1)
    # print(dia_a_dia.shape)
    tasaDescuento = [np.exp(-INTEREST_RATE * i / 365) for i in range(dia_a_dia.shape[1])]
    if cumSum:
        return np.cumsum(dia_a_dia * tasaDescuento, axis=1)
    else:
        return dia_a_dia * tasaDescuento
    



def simulacion_dif(kappa, sigma,beta,gamma, dias = N, M = 10000,seed = 0,a=6,b=2,c=2,d=0):
    # m = cantidad de caminos
    # simula n dias de los m molinos 
    if(seed != 0):
        np.random.seed(seed)
    
    molinoNorte, molinoSur, diff = np.zeros((M, dias)), np.zeros((M, dias)), np.zeros((M, dias))
    Z,W = np.random.normal(0, 1, (M, dias)), np.random.normal(0, 1, (M, dias))
    
    molinoNorte[:,0] = S0
    diff[:,0] = D0

    for t in range(1, dias):
        molinoNorte[:,t] = molinoNorte[:, t-1] + kappa * (theta2(np.zeros(M) + t,a,b,c,d) - molinoNorte[:, t-1]) * DELTA + np.sqrt(DELTA) * Z[:,t] * sigma
        diff[:,t] = diff[:, t-1] - beta * diff[:, t-1] * DELTA + np.sqrt(DELTA) * W[:,t] * gamma
        molinoSur[:,t] = molinoNorte[:,t] + diff[:,t]
    return molinoNorte , molinoSur   

def plot(a, b, c, d):
    # Parámetros iniciales
    days = 365
    M = 10000
    semillita = 42
    umbral = 7
    costo_arreglo = 250000
    i_vals = np.arange(0, days)
    
    # Calcular los nuevos valores de theta
    theta_vals = theta2(i_vals, a, b, c, d)
    
    molino_north, molino_south = simulacion_dif(kappa, sigma, beta, gamma, days, M, semillita,a,b,c,d)
    ganancia_n, ganancia_s, _, _, _, _ = ganancia(molino_north, molino_south, umbral, costo_arreglo)
    van_norte = van(ganancia_n)
    van_sur = van(ganancia_s)
    van_total = van_norte+van_sur
    
    van_tot = np.mean(van_total, axis=0)
    # Crear la figura con dos subplots
    fig, (ax2, ax1) = plt.subplots(1, 2, figsize=(15, 5))
    plt.subplots_adjust(hspace=0.4)
    
    # Gráfico de θ(ti)
    ax1.plot(i_vals, theta_vals, lw=2)
    ax1.set_title(f'θ(ti) = {a:.2f} + {b:.2f}cos({c:.2f}i + {d:.2f})')
    ax1.set_xlabel('Días')
    ax1.set_ylabel('θ(ti)')
    
    # Gráfico de VAN
    ax2.plot(i_vals[:360], van_tot[:360], color='blue', lw=2)
    ax2.set_title('Valores de VAN acumulados')
    ax2.set_xlabel('Días')
    ax2.set_ylabel('VAN')
    
    plt.show()

# Parámetros iniciales
a_init = 6
b_init = 2
c_init = 2 * np.pi / 365
d_init = 0

# Crear sliders interactivos con ipywidgets
interact(plot, 
         a=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=a_init),
         b=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=b_init),
         c=widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=c_init),
         d=widgets.FloatSlider(min=-np.pi, max=np.pi, step=0.1, value=d_init))


interactive(children=(FloatSlider(value=6.0, description='a', max=10.0, min=0.1), FloatSlider(value=2.0, descr…

<function __main__.plot(a, b, c, d)>